In [1]:
import os
from jupyter_plotly_dash import JupyterDash

import dash
# from dash import dcc
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
from flask import url_for

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "password"
shelter = AnimalShelter(username=username, password=password)


# Create dataframe from query
df = pd.DataFrame.from_records(shelter.read({}))

# Create an empty dictionary to store breed counts
breed_count = {}

# Loop through the rows in the dataframe
for index, row in df.iterrows():
    # Extract the breed name from the row
    breed = row[4]
    # If the breed is already in the dictionary, increment its count
    if breed in breed_count:
        breed_count[breed] += 1
    # If the breed is not in the dictionary, add it with a count of 1
    else:
        breed_count[breed] = 1

# Create a new dataframe using breed_count dictionary
breed_df = pd.DataFrame({'Breed': list(breed_count.keys()), 'Count': list(breed_count.values())})

# Create a pie chart using plotly express
fig = px.pie(breed_df, values='Count', names='Breed', title='Breed Counts')

#########################
# Dashboard Layout / View
#########################

app = JupyterDash("Project 2")

options = [
    {'label': 'Water Rescue', 'value': 'water_rescue'},
    {'label': 'Mountain and Wilderness Rescue', 'value': 'mountain_wilderness_rescue'},
    {'label': 'Disaster Rescue', 'value': 'disaster_rescue'},
    {'label': 'All', 'value': 'all'}
]

# Get current working directory
cur_dir = os.getcwd()

# Set image file name
img_name = 'https://imgur.com/QHgBJsa'

# Create image source path by joining directory and file name
img_src = os.path.join(cur_dir, img_name)

app.layout = html.Div([
    html.Div(className='col s12 m6',
                style={'display': 'flex', 'flexDirection': 'row', 'justifyContent': 'center', 'gap': '20px'},
                children=[
        html.H1("Austin Animal Center"),
        html.Img(src='https://i.imgur.com/QHgBJsa.png', height=80)
    ]),
    html.Div(
        className='row',
        children=[
            html.Div(
                id='datatable-div',
                className='col s12 m6',
                # TODO
                # style={'margin': '10px'},
                children=[
                    dcc.Dropdown(
                        id='dropdown',
                        options=options,
                        value='all'
                    ),
                    html.Br(),
                    dash_table.DataTable(
                        id='datatable-id',
                        columns=[
                            {'name': 'Animal Type', 'id': 'animal_type'},
                            {'name': 'Breed', 'id': 'breed'},
                            {'name': 'Color', 'id': 'color'},
                            {'name': 'Name', 'id': 'name'},
                            {'name': 'Sex', 'id': 'sex_upon_outcome'},
                            {'name': 'Latitude', 'id': 'location_lat'},
                            {'name': 'Longitude', 'id': 'location_long'},
                        ],
                        data=df.to_dict('records'),
                        filter_action="native",
                        sort_action="native",
                        sort_mode="multi",
                        page_size=10,
                        virtualization=True,
                        style_cell={'textAlign': 'left'},
                        style_table={'minWidth': '100%', 'marginBottom': '20px'},
                    )
                ]
            ),
            html.Br(),
            html.Div(
                className='col s12 m6',
                style={'display': 'flex', 'flexDirection': 'row', 'justifyContent': 'space-between'},
                children=[
                    html.Div(
                        id='map-id',
                    ),
                    dcc.Graph(
                        id='example-graph',
                        figure=fig,
                        style={'width': '900px'}
                    )
                ]
            )
        ]
    )
])

@app.callback(
    [Output(component_id='datatable-id', component_property='data'),Output(component_id='example-graph', component_property='figure')],
    [Input('dropdown', 'value')])
def filter_table(value):
    # return f'Output: {value}'
    if value == 'water_rescue':
        results = shelter.find_water_rescue({})
    elif value == 'mountain_wilderness_rescue':
        results = shelter.find_mountain_wilderness_rescue({})
    elif value == 'disaster_rescue':
        results = shelter.find_disaster_rescue({})
    else:
        results = shelter.read({})
    dff = pd.DataFrame.from_records(results)
    # Create an empty dictionary to store breed counts
    breed_count = {}

    # Loop through the rows in the dataframe
    for index, row in dff.iterrows():
        # Extract the breed name from the row
        breed = row[4]
        # If the breed is already in the dictionary, increment its count
        if breed in breed_count:
            breed_count[breed] += 1
        # If the breed is not in the dictionary, add it with a count of 1
        else:
            breed_count[breed] = 1

    # Create a new dataframe using breed_count dictionary
    breed_df = pd.DataFrame({'Breed': list(breed_count.keys()), 'Count': list(breed_count.values())})

    # Create a pie chart using plotly express
    fig = px.pie(breed_df, values='Count', names='Breed', title='Breed Counts')

    return dff.to_dict('records'), fig


#############################################
# Interaction Between Components / Controller
#############################################
# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    print('Updating Styles...')
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Callback to keep map updated 
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '400px', 'height': '300px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            html.Div([
            dl.Marker(position=[dff.iloc[i,13],dff.iloc[i,14]], children=[
                dl.Tooltip(dff.iloc[i,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[i,9])
                ])
            ]) for i in range(len(dff))])
        ])
    ]


app

KeyboardInterrupt: 